In [1]:
%load_ext autoreload
%autoreload 2}
import os
os.environ["TERRA_CONFIG_PATH"] = "/home/sabri/code/spr-21/terra_config.json"

In [2]:
import pandas as pd
import torch
import numpy as np
import clip
import spr.clip
import robustnessgym as rg
import matplotlib.pyplot as plt

In [3]:
from nltk.corpus import words
eng_words = words.words()
eng_df = pd.DataFrame({"word": eng_words})

In [19]:
(eng_df.word == "alodian").any()

True

In [20]:
from spr.clip import get_wiki_words
words_df = get_wiki_words(top_k=1e4, eng_only=True).load()

[2021-04-27 14:31:29,188][WARNING][pip._internal.operations.freeze:72] :: Could not generate requirement for distribution -illow 8.2.0 (/home/common/envs/conda/envs/rg-sabri/lib/python3.8/site-packages): Parse error at "'-illow=='": Expected W:(abcd...)
task: get_wiki_words, run_id=188


In [23]:
from spr.clip import get_wiki_words
words_df = get_wiki_words.out().load()
(words_df.word == "").any()

True

In [7]:
spr.clip.embed_words.get_runs()

,id,module,fn,run_dir,status,notebook,start_time,end_time,hostname,python_version,platform,git_commit,git_dirty,slurm_job_id
0,189,spr.clip,embed_words,/home/sabri/terra/slice/tasks/clip/embed_words...,success,False,2021-04-27 14:32:01.747755,2021-04-27 14:32:15.369195,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,a6c8ad70714c8f58e5a4361d8fa4b512e27d9040,True,None
1,186,spr.clip,embed_words,/home/sabri/terra/slice/tasks/clip/embed_words...,success,False,2021-04-27 14:18:42.721238,2021-04-27 14:20:14.387385,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,a6c8ad70714c8f58e5a4361d8fa4b512e27d9040,True,None
2,184,spr.clip,embed_words,/home/sabri/terra/slice/tasks/clip/embed_words...,success,False,2021-04-27 14:06:43.882678,2021-04-27 14:08:25.668092,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,a6c8ad70714c8f58e5a4361d8fa4b512e27d9040,True,None
3,167,spr.clip,embed_words,/home/sabri/terra/slice/tasks/clip/embed_words...,success,False,2021-04-27 04:32:09.149713,2021-04-27 04:33:50.672858,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,a6c8ad70714c8f58e5a4361d8fa4b512e27d9040,True,None
4,164,spr.clip,embed_words,/home/sabri/terra/slice/tasks/clip/embed_words...,success,False,2021-04-27 04:17:13.275109,2021-04-27 04:18:55.766138,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,a6c8ad70714c8f58e5a4361d8fa4b512e27d9040,True,None
5,163,spr.clip,embed_words,/home/sabri/terra/slice/tasks/clip/embed_words...,failure,False,2021-04-27 04:16:59.783347,2021-04-27 04:17:00.919117,pod-sabri,"3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC ...",Linux-5.4.89+-x86_64-with-glibc2.10,a6c8ad70714c8f58e5a4361d8fa4b512e27d9040,True,None


In [19]:
from spr.data.celeb import ATTRIBUTES, build_celeb_df
from spr.clip import find_explanatory_words

data_df = build_celeb_df.out(141).load()
text_embs, idx_to_word = spr.clip.embed_words.out(189, load=True)
img_embs, idx_to_img_id = spr.clip.embed_images.out(178, load=True)

df = find_explanatory_words(
    data_df=data_df,
    target_column="wearing_lipstick", 
    condition_value=0,
    condition_column="male", 
    text_embs=text_embs, 
    idx_to_word=idx_to_word, 
    img_embs=img_embs, 
    idx_to_img_id=idx_to_img_id,
    top_k=5
)
df

computing cosine similarity...


,attribute,rank,descending,r,term
0,wearing_lipstick,0,True,0.249194,accent
1,wearing_lipstick,1,True,0.234861,actress
2,wearing_lipstick,2,True,0.234157,popular
3,wearing_lipstick,3,True,0.224582,variety
4,wearing_lipstick,4,True,0.221151,did
0,wearing_lipstick,0,False,-0.329556,marking
1,wearing_lipstick,1,False,-0.322969,decisive
2,wearing_lipstick,2,False,-0.313752,divisional
3,wearing_lipstick,3,False,-0.293519,disagreed
4,wearing_lipstick,4,False,-0.293179,status


In [24]:
embs, idx_to_word = spr.clip.embed_words(
    df=spr.clip.get_wiki_words.out(188), 
    batch_size=256, 
    model="/home/common/models/clip/ViT-B-32.pt", 
    top_k=100000
)

[2021-04-27 14:32:02,338][WARNING][pip._internal.operations.freeze:72] :: Could not generate requirement for distribution -illow 8.2.0 (/home/common/envs/conda/envs/rg-sabri/lib/python3.8/site-packages): Parse error at "'-illow=='": Expected W:(abcd...)
task: embed_words, run_id=189


In [6]:
text_embs, idx_to_word = spr.clip.embed_words.out(167, load=True)

In [8]:
text_embs, idx_to_word = spr.clip.embed_images.out(168, load=True)

In [4]:
model, preprocess = clip.load(
    "/afs/cs.stanford.edu/u/sabrieyuboglu/data/models/clip/ViT-B-32.pt", device=torch.device(0), jit=False
)

In [75]:
from spr.data.celeb import build_celeb_df, celeb_transform
import torchvision.transforms as transforms

#transform = transforms.Compose([transforms.Lambda(lambda x: x.permute(2, 0, 1)), transforms.ToPILImage(), preprocess])
transform = transforms.Compose([celeb_transform, transforms.Lambda(lambda x: x.to(torch.uint8)), transforms.ToPILImage(), preprocess])
#transform = lambda x: x
df = build_celeb_df.out(29).load()
dataset = rg.Dataset.load_image_dataset(
    df[df.split == "valid"].to_dict("records"), 
    img_columns="img_path",
    transform=transform
)

In [62]:
from torchvision.datasets import CIFAR100
cifar100 = CIFAR100(root=os.path.expanduser("~/data/datasets"), download=True, train=False)

Files already downloaded and verified


In [76]:
from PIL import Image
#image = Image.open(df.iloc[250]["img_path"])
#image, class_id = cifar100[3637]
#image_input = preprocess(image).unsqueeze(0).to(0)
image_input = dataset[0]["img_path"].unsqueeze(0).to(0)
img_emb = model.encode_image(image_input)

In [77]:
text_embs = embs.load()
img_emb, text_embs = img_emb.to(0), text_embs[:10000].to(0)
img_emb /= img_emb.norm(dim=-1, keepdim=True)
text_embs /= text_embs.norm(dim=-1, keepdim=True)
similarity = (100.0 * img_emb @ text_embs.T).softmax(dim=-1)

In [78]:
vals, idxs = similarity[0].topk(20)
for val, idx in zip(list(vals), list(idxs)):
    print(f"{idx_to_word[idx]}: {val}")

a photo of actress: 0.0183868408203125
a photo of judy: 0.015716552734375
a photo of miranda: 0.01409149169921875
a photo of karen: 0.0106353759765625
a photo of lindsay: 0.00910186767578125
a photo of di: 0.00841522216796875
a photo of shannon: 0.0082855224609375
a photo of nicole: 0.0082855224609375
a photo of parks: 0.00803375244140625
a photo of tina: 0.007781982421875
a photo of sara: 0.00708770751953125
a photo of julie: 0.006977081298828125
a photo of amanda: 0.006351470947265625
a photo of jackie: 0.006256103515625
a photo of emma: 0.00615692138671875
a photo of contestant: 0.0059661865234375
a photo of sarah: 0.005695343017578125
a photo of barbara: 0.00560760498046875
a photo of anna: 0.005268096923828125
a photo of sharon: 0.005184173583984375


In [3]:
from spr.data.celeb import celeb_task_config, build_celeb_df
import imp
imp.reload(spr.clip)
out = spr.clip.embed_images(
    data_df=build_celeb_df.out(141),
    model="/home/common/models/clip/ViT-B-32.pt", 
    **celeb_task_config
)

[2021-04-27 13:51:48,272][WARNING][pip._internal.operations.freeze:72] :: Could not generate requirement for distribution -illow 8.2.0 (/home/common/envs/conda/envs/rg-sabri/lib/python3.8/site-packages): Parse error at "'-illow=='": Expected W:(abcd...)
task: embed_images, run_id=178


In [15]:
import spr.clip
from spr.vision import compute_bss
text_embs, idx_to_word = spr.clip.embed_words.out(164)
img_embs, idx_to_img_id = spr.clip.embed_images.out(165)
#components, outs, targets = compute_bss.out(run_id=271, load=True)

In [16]:
import torch
torch.load(text_embs._get_path(), map_location="cpu")

RuntimeError: Overflow when unpacking long

In [258]:
img_embs_norm = img_embs / img_embs.norm(dim=-1, keepdim=True)
text_embs_norm = text_embs / text_embs.norm(dim=-1, keepdim=True)
print('computing...')
similarity = (100.0 * img_embs_norm.to(0) @ text_embs_norm.T.to(0)).softmax(dim=-1).cpu()

computing...


In [262]:
similarity[0]

tensor([8.4639e-06, 3.6240e-05, 5.3048e-06,  ..., 2.3842e-07, 3.1590e-06,
        5.9605e-08], dtype=torch.float16)

In [263]:
vals, idxs = similarity[0].to(0).topk(20)
for val, idx in zip(list(vals), list(idxs)):
    print(f"{idx_to_word[idx]}: {val}")

a photo of janya: 0.05877685546875
a photo of fonteyn: 0.045074462890625
a photo of ellis-bextor: 0.0281982421875
a photo of delenn: 0.0176544189453125
a photo of urvashi: 0.0135345458984375
a photo of poehler: 0.00873565673828125
a photo of luan: 0.00701904296875
a photo of nyssa: 0.005641937255859375
a photo of emilie: 0.005466461181640625
a photo of geet: 0.004604339599609375
a photo of daya: 0.002971649169921875
a photo of debby: 0.0027065277099609375
a photo of mandrell: 0.002315521240234375
a photo of aga: 0.00217437744140625
a photo of tatjana: 0.0020751953125
a photo of cardinale: 0.001949310302734375
a photo of oksana: 0.0018892288208007812
a photo of natalie: 0.0018024444580078125
a photo of kelso: 0.001667022705078125
a photo of julianne: 0.0015659332275390625


In [129]:
from spr.data.celeb import build_celeb_df, celeb_transform
data_df = build_celeb_df.out(29).load()
male_labels = data_df.merge(pd.DataFrame({"file": idx_to_img_id}), on="file", how="right").male

In [63]:
y = components.max(dim=1)[0][:, 0].numpy()
indices = np.concatenate([y.argsort()[:100], y.argsort()[-100:]])

In [264]:
import statsmodels.api as sm
y = male_labels #components.max(dim=1)[0][:, 0].numpy()
x = similarity.cpu().numpy()
mod = sm.OLS(y, sm.add_constant(x[:, [
    word_to_idx["car"], word_to_idx["house"], word_to_idx["man"], word_to_idx["woman"]
]], prepend=False))
res = mod.fit()
df = pd.DataFrame(
    {
        "pvalue": res.pvalues,
        "coef": res.params,
        "coef_idx": np.arange(len(res.params)),
    }
)

In [189]:
df.sort_values("coef")

,pvalue,coef,coef_idx
x1,0.000076,-754.594721,0
x4,0.000000,-179.909968,3
x2,0.499644,-12.198507,1
const,0.000000,0.406524,4
x3,0.000000,497.059400,2


In [135]:
df[df.coef_idx == word_to_idx["man"]]

,pvalue,coef,coef_idx
x388,2.281399e-09,39.025121,387


In [136]:
df[df.coef_idx == word_to_idx["woman"]]

,pvalue,coef,coef_idx
x834,2.348174e-10,148.248955,833


In [150]:
similarity.shape

torch.Size([4949, 10000])

In [172]:
(torch.argmax(similarity[:, [word_to_idx["arm"], word_to_idx["leg"]]], dim=-1).cpu().numpy() == y).mean()

0.5595069711052738

In [171]:
similarity[:, [word_to_idx["arm"], word_to_idx["leg"]]].shape

torch.Size([4949, 2])

In [295]:
y = (components).max(dim=1)[0][:, 2].to(float)
#y = torch.tensor(male_labels.values).to(float)
indices = torch.tensor(1 - male_labels).to(bool)
corr = pearsonr(
    torch.tensor(x[indices, :50000]).to(float), y[indices],
    batch_first=False
)

for descending in [True, False]:
    sorted_vals, sorted_idxs = corr.squeeze().sort(descending=descending)
    top_k = 15
    for i, (val, idx) in enumerate(zip(sorted_vals[:top_k], sorted_idxs[:top_k])):
        print(f"{i + 1}. {idx_to_word[idx].split(' ')[-1]} ($$r={val:.3}$$)")


1. village ($$r=0.21$$)
2. sen ($$r=0.18$$)
3. hrs ($$r=0.18$$)
4. maya ($$r=0.177$$)
5. stara ($$r=0.175$$)
6. pavilions ($$r=0.174$$)
7. townships ($$r=0.172$$)
8. pavilion ($$r=0.171$$)
9. website ($$r=0.168$$)
10. showcased ($$r=0.168$$)
11. nikita ($$r=0.168$$)
12. ravine ($$r=0.168$$)
13. canvas ($$r=0.167$$)
14. popularly ($$r=0.167$$)
15. watermill ($$r=0.167$$)
1. corrado ($$r=-0.237$$)
2. callus ($$r=-0.219$$)
3. mandals ($$r=-0.211$$)
4. trouser ($$r=-0.21$$)
5. zoologist ($$r=-0.209$$)
6. skid ($$r=-0.207$$)
7. planter ($$r=-0.206$$)
8. leiber ($$r=-0.206$$)
9. rips ($$r=-0.204$$)
10. guido ($$r=-0.203$$)
11. travers ($$r=-0.202$$)
12. casimir ($$r=-0.2$$)
13. memo ($$r=-0.199$$)
14. choate ($$r=-0.199$$)
15. curt ($$r=-0.199$$)


In [254]:
y[indices]

tensor([36.3641, 16.7326, 27.8381,  ..., 20.7277, 23.2033, 20.4553])

In [197]:
from scipy.stats import pearsonr
for word in ["man", "woman"]:
    r = pearsonr(x[:, word_to_idx[word]], y)
    print(r)

(0.7805339144071342, 0.0)
(-0.7107771793399529, 0.0)


In [217]:
def pearsonr(x, y, batch_first=True):
    r"""Computes Pearson Correlation Coefficient across rows.
    Pearson Correlation Coefficient (also known as Linear Correlation
    Coefficient or Pearson's :math:`\rho`) is computed as:
    .. math::
        \rho = \frac {E[(X-\mu_X)(Y-\mu_Y)]} {\sigma_X\sigma_Y}
    If inputs are matrices, then then we assume that we are given a
    mini-batch of sequences, and the correlation coefficient is
    computed for each sequence independently and returned as a vector. If
    `batch_fist` is `True`, then we assume that every row represents a
    sequence in the mini-batch, otherwise we assume that batch information
    is in the columns.
    Warning:
        We do not account for the multi-dimensional case. This function has
        been tested only for the 2D case, either in `batch_first==True` or in
        `batch_first==False` mode. In the multi-dimensional case,
        it is possible that the values returned will be meaningless.
    Args:
        x (torch.Tensor): input tensor
        y (torch.Tensor): target tensor
        batch_first (bool, optional): controls if batch dimension is first.
            Default: `True`
    Returns:
        torch.Tensor: correlation coefficient between `x` and `y`
    Note:
        :math:`\sigma_X` is computed using **PyTorch** builtin
        **Tensor.std()**, which by default uses Bessel correction:
        .. math::
            \sigma_X=\displaystyle\frac{1}{N-1}\sum_{i=1}^N({x_i}-\bar{x})^2
        We therefore account for this correction in the computation of the
        covariance by multiplying it with :math:`\frac{1}{N-1}`.
    Shape:
        - Input: :math:`(N, M)` for correlation between matrices,
          or :math:`(M)` for correlation between vectors
        - Target: :math:`(N, M)` or :math:`(M)`. Must be identical to input
        - Output: :math:`(N, 1)` for correlation between matrices,
          or :math:`(1)` for correlation between vectors
    Examples:
        >>> import torch
        >>> _ = torch.manual_seed(0)
        >>> input = torch.rand(3, 5)
        >>> target = torch.rand(3, 5)
        >>> output = pearsonr(input, target)
        >>> print('Pearson Correlation between input and target is {0}'.format(output[:, 0]))
        Pearson Correlation between input and target is tensor([ 0.2991, -0.8471,  0.9138])
    """  # noqa: E501

    if len(x.shape) - len(y.shape) == 1:
        y = y.unsqueeze(-1)

    if batch_first:
        dim = -1
    else:
        dim = 0

    centered_x = x - x.mean(dim=dim, keepdim=True)
    centered_y = y - y.mean(dim=dim, keepdim=True)

    covariance = (centered_x * centered_y).sum(dim=dim, keepdim=True)

    bessel_corrected_covariance = covariance / (x.shape[dim] - 1)

    x_std = x.std(dim=dim, keepdim=True)
    y_std = y.std(dim=dim, keepdim=True)

    corr = bessel_corrected_covariance / (x_std * y_std)

    return corr

In [209]:
a, b = torch.tensor(x[:, [word_to_idx["man"], word_to_idx["woman"]]]).to(float), torch.tensor(y).to(float)

In [216]:
a * b.unsqueeze(-1)

tensor([[0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [1.5564e-03, 1.0967e-05],
        ...,
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00]], dtype=torch.float64)

In [212]:
b.shape

torch.Size([4949])

In [213]:
a.shape

torch.Size([4949, 2])

In [310]:
from spr.data.celeb import ATTRIBUTES
from spr.data.celeb import build_celeb_df, celeb_transform
data_df = build_celeb_df.out(29).load()
top_k = 10


for attribute in ATTRIBUTES:
    y = torch.tensor(data_df.merge(pd.DataFrame({"file": idx_to_img_id}), on="file", how="right")[attribute]).to(float)
    corr = pearsonr(
        torch.tensor(x[:, :50000]).to(float), y,
        batch_first=False
    )
    #print(f"Terms describing `{attribute}`")  
    dfs = []
    for descending in [True, False]:
        #print(f"{'Top' if descending else 'Bottom'} terms")
        sorted_vals, sorted_idxs = corr.squeeze().sort(descending=descending)
        dfs.append(pd.DataFrame({
            "attribute": attribute,
            "rank": np.arange(top_k),
            "descending": descending,
            "r": sorted_vals[:top_k],
            "term": [idx_to_word[idx] for idx in sorted_idxs[:top_k]]
        }))
        for i, (val, idx) in enumerate(zip(sorted_vals[:top_k], sorted_idxs[:top_k])):
            #print(f"{i + 1}. {idx_to_word[idx].split(' ')[-1]} ($$r={val:.3}$$)")
            pass
    df = pd.concat(dfs)
    #print("\n\n")


SyntaxError: closing parenthesis '}' does not match opening parenthesis '[' on line 23 (<ipython-input-310-572d9083a903>, line 24)

In [309]:
df

,attribute,rank,descending,term
0,young,0,True,0.418772
1,young,1,True,0.408523
2,young,2,True,0.403880
3,young,3,True,0.395321
4,young,4,True,0.395139
5,young,5,True,0.392552
6,young,6,True,0.391520
7,young,7,True,0.388417
8,young,8,True,0.387592
9,young,9,True,0.384040


In [9]:
import terra
terra.out(177, load=True)

,attribute,rank,descending,r,term
0,5_o_clock_shadow,0,True,0.052942,acf
1,5_o_clock_shadow,1,True,0.052239,purports
2,5_o_clock_shadow,2,True,0.050676,env
3,5_o_clock_shadow,3,True,0.050207,mapper
4,5_o_clock_shadow,4,True,0.049232,exacted
...,...,...,...,...,...
795,young,5,False,-0.048642,subtribe
796,young,6,False,-0.048283,bank's
797,young,7,False,-0.048271,baiju
798,young,8,False,-0.048115,yandex


In [25]:
import terra
terra.get_run_dir(190)

'/home/sabri/terra/slice/tasks/__main__/evaluate_clip_on_ground_truth_slices/_runs/190'